In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from rfm import NTKModel
import torch.nn.functional as F
import logging

# Cấu hình logging: ghi log vào file và in ra console
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(message)s')

# File handler: ghi log vào output.txt
file_handler = logging.FileHandler("output.txt")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Console handler: in log ra console
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# Hàm chuyển đổi batch: chuyển đổi nhãn thành one-hot encoding với 10 lớp
def one_hot_collate(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, 0)
    labels = torch.tensor(labels)
    # Chuyển đổi nhãn sang one-hot (float)
    labels = F.one_hot(labels, num_classes=10).float()
    return images, labels

# Thiết lập thiết bị và bộ nhớ GPU (nếu có)
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    DEV_MEM_GB = torch.cuda.get_device_properties(DEVICE).total_memory // 1024**3 - 1
else:
    DEVICE = torch.device("cpu")
    DEV_MEM_GB = 8

# Định nghĩa transform: chuyển đổi ảnh MNIST thành tensor và làm phẳng thành vector 784 chiều
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Tải dataset MNIST cho training và testing
full_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Chia tập train ra thành một phần nhỏ để tránh làm đầy bộ nhớ
subset_size = 10000  # Có thể điều chỉnh theo khả năng của máy
train_subset, _ = random_split(full_train_dataset, [subset_size, len(full_train_dataset) - subset_size])

# Tạo DataLoader cho tập train và test, sử dụng collate_fn để chuyển đổi nhãn
batch_size = 16
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=one_hot_collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=one_hot_collate)

# Trích xuất dữ liệu từ DataLoader
X_train, y_train = next(iter(DataLoader(train_subset, batch_size=len(train_subset), collate_fn=one_hot_collate)))
X_test, y_test = next(iter(DataLoader(test_dataset, batch_size=len(test_dataset), collate_fn=one_hot_collate)))

# Khởi tạo NTKModel
ntk_model = NTKModel(
    sqrtM=torch.eye(X_train.shape[1]),  # Ma trận sqrtM (có thể điều chỉnh)
    device=DEVICE
)

# Huấn luyện NTKModel
logger.info("Training NTKModel")
ntk_model.fit(
    X=X_train.to(DEVICE),
    y=y_train.to(DEVICE),
    reg=1e-3  # Tham số regularization
)

# Dự đoán trên tập test
predictions = ntk_model.predict(X_test.to(DEVICE))

# Đánh giá kết quả
accuracy = (predictions.argmax(dim=1) == y_test.to(DEVICE).argmax(dim=1)).float().mean().item()
logger.info(f"Test Accuracy: {accuracy:.4f}")

2025-03-26 23:55:23,278 - Training NTKModel
2025-03-26 23:55:23,278 - Training NTKModel
2025-03-26 23:55:23,278 - Training NTKModel
2025-03-26 23:55:23,278 - Training NTKModel
2025-03-26 23:55:23,278 - Training NTKModel
2025-03-26 23:55:23,278 - Training NTKModel
2025-03-26 23:55:23,278 - Training NTKModel
2025-03-26 23:55:23,278 - Training NTKModel


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [7]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    DEV_MEM_GB = torch.cuda.get_device_properties(DEVICE).total_memory // 1024**3 - 1
else:
    DEVICE = torch.device("cpu")
    DEV_MEM_GB = 8

In [8]:
print(DEVICE, DEV_MEM_GB)

cuda 2
